## Carregamento dos dados

In [3]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

X_ = torch.load('../data/processed/X_.pt')
y_ = torch.load('../data/processed/y_.pt')

real_set = TensorDataset(X_, y_)

X_0 = torch.load('../data/processed/X_0.pt')
X_1 = torch.load('../data/processed/X_1.pt')
X_2 = torch.load('../data/processed/X_2.pt')
X_3 = torch.load('../data/processed/X_3.pt')

X_0 = torch.from_numpy(np.expand_dims(X_0, axis=1))
X_1 = torch.from_numpy(np.expand_dims(X_1, axis=1))
X_2 = torch.from_numpy(np.expand_dims(X_2, axis=1))
X_3 = torch.from_numpy(np.expand_dims(X_3, axis=1))

gen_set_0 = TensorDataset(X_0, torch.as_tensor(np.full(1296, 0)))
gen_set_1 = TensorDataset(X_1, torch.as_tensor(np.full(1296, 1)))
gen_set_2 = TensorDataset(X_2, torch.as_tensor(np.full(1296, 2)))
gen_set_3 = TensorDataset(X_3, torch.as_tensor(np.full(1296, 3)))

print(X_0.shape)
print(X_1.shape)
print(X_2.shape)
print(X_3.shape)
print(X_.shape)
print(y_.shape)

torch.Size([1296, 1, 22, 1125])
torch.Size([1296, 1, 22, 1125])
torch.Size([1296, 1, 22, 1125])
torch.Size([1296, 1, 22, 1125])
torch.Size([5184, 22, 1125])
torch.Size([5184])


In [2]:
from torch.utils.data import random_split

gen_set_0_tr, gen_set_0_vl = random_split(gen_set_0, [648, 648])
gen_set_1_tr, gen_set_1_vl = random_split(gen_set_1, [648, 648])
gen_set_2_tr, gen_set_2_vl = random_split(gen_set_2, [648, 648])
gen_set_3_tr, gen_set_3_vl = random_split(gen_set_3, [648, 648])

In [3]:
gen_loader_0_tr = DataLoader(dataset=gen_set_0_tr, batch_size=648, shuffle=True)
gen_loader_0_vl = DataLoader(dataset=gen_set_0_vl, batch_size=648, shuffle=True)
gen_loader_1_tr = DataLoader(dataset=gen_set_1_tr, batch_size=648, shuffle=True)
gen_loader_1_vl = DataLoader(dataset=gen_set_1_vl, batch_size=648, shuffle=True)
gen_loader_2_tr = DataLoader(dataset=gen_set_2_tr, batch_size=648, shuffle=True)
gen_loader_2_vl = DataLoader(dataset=gen_set_2_vl, batch_size=648, shuffle=True)
gen_loader_3_tr = DataLoader(dataset=gen_set_3_tr, batch_size=648, shuffle=True)
gen_loader_3_vl = DataLoader(dataset=gen_set_3_vl, batch_size=648, shuffle=True)

## Definição do modelo do VAE

In [4]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [5]:
class CNN_VAE(nn.Module):
    def __init__(self):
        super(CNN_VAE, self).__init__()
 
        self.enc1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=4, stride=2, padding=1)
        self.enc2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=4, stride=2, padding=1)
        self.enc3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.enc4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=2, stride=2, padding=0)
        
        self.fc1 = nn.Linear(64, 128)
        self.fc_mu = nn.Linear(128, 4)
        self.fc_log_var = nn.Linear(128, 4)
        self.fc2 = nn.Linear(4, 64)
        
        self.dec1 = nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=(1, 140), stride=2, padding=0)
        self.dec2 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=(4, 4), stride=2, padding=0)
        self.dec3 = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=(4, 1), stride=2, padding=0)
        self.dec4 = nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=(4, 1), stride=2, padding=0)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        sample = mu + (eps * std)
        return sample
 
    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        batch, _, _, _ = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        hidden = self.fc1(x)
        
        mu = self.fc_mu(hidden)
        log_var = self.fc_log_var(hidden)
        
        z = self.reparameterize(mu, log_var)
        z = self.fc2(z)
        z = z.view(-1, 64, 1, 1)

        x = F.relu(self.dec1(z))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        reconstruction = torch.sigmoid(self.dec4(x))
        return reconstruction, mu, log_var

In [6]:
def final_loss(bce_loss, mu, logvar):
    BCE = bce_loss
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [7]:
def train(model, dataloader, dataset, device, optimizer, criterion):
    model.train()
    running_loss = 0.0
    counter = 0
    for i, data in tqdm(enumerate(dataloader), total=int(len(dataset)/dataloader.batch_size)):
        counter += 1
        data = data[0]
        data = data.to(device)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        bce_loss = criterion(reconstruction, data)
        loss = final_loss(bce_loss, mu, logvar)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    train_loss = running_loss / counter 
    return train_loss

In [8]:
def validate(model, dataloader, dataset, device, criterion):
    model.eval()
    running_loss = 0.0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(dataset)/dataloader.batch_size)):
            counter += 1
            data= data[0]
            data = data.to(device)
            reconstruction, mu, logvar = model(data)
            bce_loss = criterion(reconstruction, data)
            loss = final_loss(bce_loss, mu, logvar)
            running_loss += loss.item()

            if i == int(len(dataset)/dataloader.batch_size) - 1:
                fake = reconstruction
    val_loss = running_loss / counter
    return val_loss, fake

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN_VAE().to(device)

In [10]:
from torchsummary import summary
summary(model, (1, 22, 1125))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 8, 11, 562]             136
            Conv2d-2           [-1, 16, 5, 281]           2,064
            Conv2d-3           [-1, 32, 2, 140]           8,224
            Conv2d-4            [-1, 64, 1, 70]           8,256
            Linear-5                  [-1, 128]           8,320
            Linear-6                    [-1, 4]             516
            Linear-7                    [-1, 4]             516
            Linear-8                   [-1, 64]             320
   ConvTranspose2d-9           [-1, 64, 1, 140]         573,504
  ConvTranspose2d-10           [-1, 32, 4, 282]          32,800
  ConvTranspose2d-11          [-1, 16, 10, 563]           2,064
  ConvTranspose2d-12          [-1, 1, 22, 1125]              65
Total params: 636,785
Trainable params: 636,785
Non-trainable params: 0
-------------------------------

In [10]:
lr = 0.001
epochs = 100
batch_size = 648
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss(reduction='sum')

In [11]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = train(
        model, gen_loader_0_tr, gen_set_0_tr, device, optimizer, criterion
    )
    valid_epoch_loss, fake = validate(
        model, gen_loader_0_vl, gen_set_0_tr, device, criterion
    )
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {valid_epoch_loss:.4f}")
print('TRAINING COMPLETE')

Epoch 1 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.13it/s]


Train Loss: 7718763.5000
Val Loss: 7702124.0000
Epoch 2 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.43it/s]


Train Loss: 7701984.5000
Val Loss: 7685239.0000
Epoch 3 of 100


...

Train Loss: -13636005.0000
Val Loss: -4018564.0000
Epoch 99 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.31it/s]


Train Loss: -13640317.0000
Val Loss: -4145148.5000
Epoch 100 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.43it/s]

Train Loss: -13972654.0000
Val Loss: -4091591.5000
TRAINING COMPLETE


In [12]:
print(fake.shape)

torch.Size([648, 1, 22, 1125])


In [13]:
size=(22, 1125)
fake_0 = fake.detach().cpu().view(-1, *size)
print(fake_0.shape)

torch.Size([648, 22, 1125])


In [14]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [15]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = train(
        model, gen_loader_1_tr, gen_set_1_tr, device, optimizer, criterion
    )
    valid_epoch_loss, fake = validate(
        model, gen_loader_1_vl, gen_set_1_vl, device, criterion
    )
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {valid_epoch_loss:.4f}")
print('TRAINING COMPLETE')

Epoch 1 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.46it/s]


Train Loss: -1003270.4375
Val Loss: -2179570.7500
Epoch 2 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.42it/s]


Train Loss: -933639.6875
Val Loss: -1911937.0000
Epoch 3 of 100


...

Train Loss: -12183919.0000
Val Loss: -7596565.0000
Epoch 99 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.34it/s]


Train Loss: -13378503.0000
Val Loss: -6618841.0000
Epoch 100 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.31it/s]

Train Loss: -11619170.0000
Val Loss: -7987339.5000
TRAINING COMPLETE


In [16]:
print(fake.shape)

torch.Size([648, 1, 22, 1125])


In [17]:
fake_1 = fake.detach().cpu().view(-1, *size)
print(fake_1.shape)

torch.Size([648, 22, 1125])


In [18]:
gc.collect()
torch.cuda.empty_cache()

In [19]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = train(
        model, gen_loader_2_tr, gen_set_2_tr, device, optimizer, criterion
    )
    valid_epoch_loss, fake = validate(
        model, gen_loader_2_vl, gen_set_2_vl, device, criterion
    )
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {valid_epoch_loss:.4f}")
print('TRAINING COMPLETE')

Epoch 1 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.03it/s]


Train Loss: -9645053.0000
Val Loss: -6430198.0000
Epoch 2 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.24it/s]


Train Loss: -9988277.0000
Val Loss: -5723174.5000
Epoch 3 of 100


...

Train Loss: -18328176.0000
Val Loss: -7284254.5000
Epoch 99 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.99it/s]


Train Loss: -18174990.0000
Val Loss: -6673626.0000
Epoch 100 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.36it/s]

Train Loss: -17005472.0000
Val Loss: -6695204.5000
TRAINING COMPLETE


In [20]:
print(fake.shape)

torch.Size([648, 1, 22, 1125])


In [21]:
fake_2 = fake.detach().cpu().view(-1, *size)
print(fake_2.shape)

torch.Size([648, 22, 1125])


In [22]:
gc.collect()
torch.cuda.empty_cache()

In [23]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = train(
        model, gen_loader_3_tr, gen_set_3_tr, device, optimizer, criterion
    )
    valid_epoch_loss, fake = validate(
        model, gen_loader_3_vl, gen_set_3_vl, device, criterion
    )
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {valid_epoch_loss:.4f}")
print('TRAINING COMPLETE')

Epoch 1 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.26it/s]


Train Loss: -6054501.5000
Val Loss: -5467566.0000
Epoch 2 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.10it/s]


Train Loss: -5521248.5000
Val Loss: -4647273.5000
Epoch 3 of 100


...

Train Loss: -16882272.0000
Val Loss: -7923467.5000
Epoch 99 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.35it/s]


Train Loss: -14975075.0000
Val Loss: -7615266.5000
Epoch 100 of 100


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.34it/s]

Train Loss: -14297049.0000
Val Loss: -7857430.0000
TRAINING COMPLETE


In [24]:
print(fake.shape)

torch.Size([648, 1, 22, 1125])


In [25]:
fake_3 = fake.detach().cpu().view(-1, *size)
print(fake_3.shape)

torch.Size([648, 22, 1125])


In [26]:
model.cpu()
del model
gc.collect()
torch.cuda.empty_cache()

## Processamento dos dados sintéticos para classificação

In [34]:
fake = torch.cat((fake_0, fake_1, fake_2, fake_3), 0)
y_fake = torch.cat((torch.as_tensor(np.full(648, 0)), torch.as_tensor(np.full(648, 1)), torch.as_tensor(np.full(648, 2)), torch.as_tensor(np.full(648, 3))), 0)
print(fake.shape)
print(y_fake.shape)

torch.Size([2592, 22, 1125])
torch.Size([2592])


In [36]:
fake_set = TensorDataset(fake, y_fake)

In [38]:
fake_fulltrainset, fake_evalset = random_split(fake_set, [1296, 1296])
fake_trainset, fake_testset = random_split(fake_fulltrainset, [864, 432])
real_fulltrainset, real_evalset = random_split(real_set, [2592, 2592])
real_trainset, real_testset = random_split(real_fulltrainset, [1728, 864])

## Definição do modelo do classificador

In [39]:
from braindecode.util import set_random_seeds
from braindecode.models import EEGNetv4

cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = False

seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
n_chans = 22
input_window_samples = 1125
F1, D = 4, 2
kernel_length = 64

model = EEGNetv4(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
    F1=8,
    D=2,
    F2=F1*D,
    kernel_length=kernel_length,
    drop_prob=0.5
)
model.to(device);

## Treinamento do classificador com dados reais

In [41]:
from skorch.helper import predefined_split
from skorch.callbacks import LRScheduler
from braindecode import EEGClassifier

batch_size = 32
n_epochs = 50

real_clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.Adam,
    train_split=predefined_split(real_testset),
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)
real_clf.fit(real_trainset, y=None, epochs=n_epochs);

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.3721        1.4100            0.2951        1.3749  0.0100  0.7727
      2            0.4340        1.3273            0.3657        1.3189  0.0100  0.6615
      3            0.4664        1.2894            0.3947        1.2728  0.0100  0.6793
      4            0.4456        1.2406            0.3831        1.4101  0.0099  0.6600
      5            0.5231        1.2098            0.4410        1.1999  0.0098  0.6583
      6            0.5561        1.1784            0.4583        1.1863  0.0097  0.6568
      7            0.5365        1.1753            0.4618        1.1948  0.0096  0.6585
      8            0.5341        1.1579            0.4225        1.2789  0.0095  0.6598
      9            0.5914        1.1508            0.4664        1.1888  0.0094  0.6642
     10            0.5359       

In [42]:
print(f"Mean Accuracy: {np.mean(real_clf.predict(real_evalset) == [y for X,y in real_evalset])*100:.2f}%")

Mean Accuracy: 60.96%


In [43]:
print(f"Mean Accuracy: {np.mean(real_clf.predict(fake_evalset) == [y for X,y in fake_evalset])*100:.2f}%")

Mean Accuracy: 24.77%


## Treinamento do classificador com dados sintéticos

In [44]:
fake_clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.Adam,
    train_split=predefined_split(fake_testset),
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)
fake_clf.fit(fake_trainset, y=None, epochs=n_epochs);

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.2593        0.2146            0.2292        1.2707  0.0100  0.3603
      2            0.3021        0.0042            0.2847        1.1408  0.0100  0.3366
      3            0.2998        0.0009            0.2639        1.1193  0.0100  0.3459
      4            0.2963        0.0007            0.2569        1.1023  0.0099  0.3430
      5            0.3310        0.0007            0.3032        1.0433  0.0098  0.3392
      6            0.3634        0.0004            0.3472        0.9780  0.0097  0.3358
      7            0.4225        0.0003            0.4190        0.9067  0.0096  0.3340
      8            0.5289        0.0003            0.4907        0.8312  0.0095  0.3403
      9            0.9132        0.0003            0.8657        0.7136  0.0094  0.3352
     10            1.0000       

In [45]:
print(f"Mean Accuracy: {np.mean(fake_clf.predict(real_evalset) == [y for X,y in real_evalset])*100:.2f}%")

Mean Accuracy: 32.02%


In [46]:
print(f"Mean Accuracy: {np.mean(fake_clf.predict(fake_evalset) == [y for X,y in fake_evalset])*100:.2f}%")

Mean Accuracy: 100.00%


## Distância euclidiana entre os dados reais e sintéticos

In [48]:
size=(22, 1125)
real = X_[:2592].view(-1, *size)

In [49]:
torch.norm(real-fake, 2)

tensor(23613.5742)